In [1]:
import argparse
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.utils.data
import torchvision.utils as vutils
import pandas as pd
from math import pi, sin, cos, sqrt, log
from toolz import partial, curry
from torch import Tensor
from torch import nn, optim, distributions
from torchvision import datasets, transforms, models
from torchvision.utils import save_image, make_grid
from typing import Callable, Iterator, Union, Optional, TypeVar
from typing import List, Set, Dict, Tuple
from typing import Mapping, MutableMapping, Sequence, Iterable
from typing import Union, Any, cast
# my own sauce
from my_torch_utils import denorm, normalize, mixedGaussianCircular
from my_torch_utils import fclayer, init_weights
from my_torch_utils import plot_images, save_reconstructs, save_random_reconstructs
from my_torch_utils import fnorm, replicate, logNorm
from my_torch_utils import scsimDataset
import scsim.scsim as scsim

In [3]:
### generate new simulated dataset
simulator = scsim.scsim(ngenes=5000, nproggenes=300, ncells=10000,
        ngroups=10, seed=42,libloc=7.64, libscale=0.78,
        mean_rate=7.68, mean_shape=0.34, expoutprob=0.00286, expoutloc=6.15,
        expoutscale=0.49, 
        )

ngenes = 10**4
ncells = 10**4
K=10
deprob = .025
progdeloc = deloc = deval = 1
descale = 1.0
progcellfrac = .35
deprob = .025
doubletfrac = .0
ndoublets=int(doubletfrac*ncells)
nproggenes = 400
nproggroups = int(K/3)
proggroups = list(range(1, nproggroups+1))
randseed = 42

simulator = scsim.scsim(
    ngenes=ngenes,
    ncells=ncells,
    ngroups=K,
    libloc=7.64,
    libscale=0.78,
    mean_rate=7.68,
    mean_shape=0.34,
    expoutprob=0.00286,
    expoutloc=6.15,
    expoutscale=0.49,
    diffexpprob=deprob,
    diffexpdownprob=0.0,
    diffexploc=deloc,
    diffexpscale=descale,
    bcv_dispersion=0.448,
    bcv_dof=22.087,
    ndoublets=ndoublets,
    nproggenes=nproggenes,
    progdownprob=0.0,
    progdeloc=progdeloc,
    progdescale=descale,
    progcellfrac=progcellfrac,
    proggoups=proggroups,
    minprogusage=0.1,
    maxprogusage=0.7,
    seed=randseed,
    
)

simulator.simulate()


Simulating cells
Simulating gene params
Simulating program
Simulating DE
Simulating cell-gene means
   - Getting mean for activity program carrying cells
   - Getting mean for non activity program carrying cells
   - Normalizing by cell libsize
Adjusting means
Simulating counts


In [4]:
scsim.save_df(simulator.cellparams, "data/scrnasim/cellparams")
scsim.save_df(simulator.counts, "data/scrnasim/counts")
scsim.save_df(simulator.geneparams, "data/scrnasim/geneparams")

In [7]:
# prepare train/test sets
device = "cuda" if torch.cuda.is_available() else "cpu"
dataSet = scsimDataset("data/scrnasim/counts.npz",
        "data/scrnasim/cellparams.npz")
trainD, testD = dataSet.__train_test_split__(8500)

trainLoader = torch.utils.data.DataLoader(trainD, batch_size=128, shuffle=True)
testLoader = torch.utils.data.DataLoader(testD, batch_size=128, shuffle=True)


In [8]:
# define the model
class Classifier(nn.Module):
    def __init__(self, nin=10**4, nh=10**3, nclasses=10):
        super(Classifier, self).__init__()
        self.nclasses = nclasses
        self.nin = nin
        self.main = nn.Sequential(
                fclayer(nin=nin, nout=nh, batchnorm=True, dropout=0.2,
                    activation=nn.LeakyReLU(),),
                nn.Linear(nh, nclasses),
                nn.LogSoftmax(dim=1),
                )
        return

    def forward(self,x):
        logp = self.main(x)
        return logp


In [10]:
criterion = nn.NLLLoss(reduction="mean")
model = Classifier().to(device)
model.apply(init_weights)
optimizer = optim.Adam(model.parameters())

In [11]:
for epoch in range(15):
    for idx, (data, labels) in enumerate(trainLoader):
        x = data.to(device)
        target = (labels - 1).to(device)
        model.train()
        model.zero_grad()
        logprob = model(x)
        loss = criterion(logprob, target)
        loss.backward()
        optimizer.step()
        if idx % 100 == 0:
            print(
                "loss:\n",
                loss.item(),
                )


loss:
 2.54552960395813
loss:
 0.09729622304439545
loss:
 0.03515252470970154
loss:
 0.026659106835722923
loss:
 0.008766941726207733
loss:
 0.014332371763885021
loss:
 0.008803398348391056
loss:
 0.0030491738580167294
loss:
 0.00130040745716542
loss:
 0.007098771166056395
loss:
 0.02561776712536812
loss:
 0.0033673604484647512
loss:
 0.0013898734468966722
loss:
 0.0015989248640835285
loss:
 0.004438658244907856


In [13]:
# testing
xs, ls = testLoader.__iter__().next()

# convert the labels to 0-indexing
ls -= 1

model.to("cpu")
ys = model(xs)
probs = ys.exp()
print("loss = ", criterion(ys, ls).item())


loss =  0.14314091205596924


In [16]:
predicts = probs.argmax(axis=1)
print("prdicted classes: \n", 
      predicts,
      "\nactual classes: \n",
      ls,
     )

prdicted classes: 
 tensor([6, 6, 5, 3, 3, 2, 4, 8, 7, 1, 6, 8, 4, 5, 9, 4, 3, 7, 0, 7, 0, 2, 8, 2,
        3, 1, 0, 8, 9, 3, 2, 7, 3, 3, 1, 2, 2, 6, 7, 0, 0, 2, 4, 7, 6, 6, 9, 3,
        2, 8, 5, 7, 9, 3, 7, 3, 1, 7, 7, 8, 9, 5, 1, 3, 9, 9, 9, 5, 9, 7, 1, 5,
        6, 9, 2, 3, 0, 4, 4, 5, 0, 0, 2, 2, 7, 6, 9, 7, 4, 0, 4, 4, 4, 1, 3, 2,
        0, 0, 9, 0, 1, 9, 3, 5, 6, 7, 0, 8, 9, 3, 1, 7, 2, 4, 2, 5, 6, 9, 3, 2,
        8, 3, 0, 5, 6, 7, 9, 2]) 
actual classes: 
 tensor([6, 6, 5, 3, 3, 2, 4, 8, 7, 1, 6, 8, 4, 5, 9, 4, 3, 7, 0, 7, 0, 2, 8, 2,
        3, 1, 0, 8, 9, 3, 2, 7, 3, 3, 1, 4, 2, 6, 7, 0, 0, 2, 4, 7, 6, 6, 9, 3,
        2, 8, 5, 7, 9, 3, 7, 3, 1, 7, 7, 8, 9, 5, 1, 3, 9, 9, 9, 5, 9, 7, 1, 5,
        6, 9, 2, 3, 2, 4, 4, 5, 0, 0, 2, 2, 7, 6, 9, 7, 4, 0, 4, 4, 4, 1, 3, 2,
        0, 0, 9, 0, 1, 9, 3, 5, 4, 7, 0, 8, 9, 3, 1, 7, 2, 4, 2, 5, 6, 9, 3, 1,
        8, 3, 5, 5, 1, 7, 9, 2])


In [18]:
(predicts == ls)

tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True, False,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True, False,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True, False,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True, False,
         True,  True, False,  True, False,  True,  True,  True])

In [20]:
accuracy = (predicts == ls).sum() / len(ls)
accuracy


tensor(0.9531)